In [15]:
import pandas as pd
import xgboost as xgb
from rdkit import Chem
from rdkit.Chem import Descriptors
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from sklearn import neural_network as nn
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, KFold
import numpy as np
import matplotlib.pyplot as plt

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [16]:
df = pd.read_csv("Permuted Data")
df

,Standardized_SMILES,pchembl_value,MolWt,LogP,QED
0,CC(C)c1nnc2ccc(-c3c[nH]nc3-c3cc(F)ccc3F)cn12,8.55,339.349,4.18800,0.608797
1,CC(=O)Nc1cnccc1CCOc1ccc(NC(=O)N=c2cc(C(C)(C)C)...,8.10,576.701,6.67222,0.200740
2,Cc1ccc(-n2[nH]c(C(C)(C)C)cc2=NC(=O)Nc2ccc(OCCN...,7.81,527.669,5.40832,0.351338
3,O=c1ncc2c(-c3ccccc3Cl)cc(C(O)N3CCNCC3)cc2n1-c1...,8.82,515.828,4.90900,0.404159
4,Cn1c(-c2cn(C(C)(C)C(=O)O)nn2)nc(-c2ccc(F)cc2)c...,5.83,406.421,3.36640,0.545632
...,...,...,...,...,...
1440,CCN(C(=O)c1csc2ccccc12)c1cc[nH]c(=NC(C)c2ccccc...,6.73,402.523,4.95300,0.513182
1441,COc1cc2[nH]cnc(=Nc3cc(NC(=O)c4ccnc(N5CCOCC5)c4...,7.30,597.720,4.46092,0.290574
1442,CN1CCC(c2cc(-c3ccc(F)cc3Cl)c3c(c2)N(c2c(Cl)ccc...,9.30,518.847,7.62340,0.386913
1443,Cc1ccc(C(=O)Nc2cccc(C(F)(F)F)c2C)cc1-c1ccc2[nH...,8.05,547.625,6.50444,0.289316


In [17]:
import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt


# Create a DeepChem dataset
tasks = ["pchembl_value"]
featurizer = dc.feat.CircularFingerprint(size=2048)
loader = dc.data.CSVLoader(tasks=tasks, feature_field="Standardized_SMILES", featurizer=featurizer)
dataset = loader.create_dataset("Permuted Data")

# Define the scaffold splitter
splitter = dc.splits.RandomSplitter()

# Get scaffold split indices
train_valid_test = splitter.k_fold_split(dataset, k=10, seed=42)


In [18]:
#MLP

import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = nn.MLPRegressor(hidden_layer_sizes=(min(25,round(np.sqrt(X_train.shape[0])),)), random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1301, Test size: 144


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 1 Accuracy: (0.8572948682266512, 8.842437180723823e-43)
Fold 1 rmse: 0.6695943000606824
Fold 1 r_squared: 0.7296698057224584
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.7783728009562503
Fold 2 - Train size: 1301, Test size: 144


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 2 Accuracy: (0.8064090981419971, 3.308594589747797e-34)
Fold 2 rmse: 0.6904634947806206
Fold 2 r_squared: 0.6019953285410264
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.7706844560884062
Fold 3 - Train size: 1301, Test size: 144


...

Fold 8 Accuracy: (0.8599713222024427, 1.294767044632769e-43)
Fold 8 rmse: 0.6168038250271651
Fold 8 r_squared: 0.7307499652785293
Fold 8 Mean of Two Highest Tanimoto Similarities: 0.7724352624398779
Fold 9 - Train size: 1300, Test size: 145


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 9 Accuracy: (0.792486574778077, 1.635555704936788e-32)
Fold 9 rmse: 0.663621550822865
Fold 9 r_squared: 0.5330389757605122
Fold 9 Mean of Two Highest Tanimoto Similarities: 0.7686215386286485
Fold 10 - Train size: 1300, Test size: 145


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 10 Accuracy: (0.8493105730989189, 1.6497569098980554e-41)
Fold 10 rmse: 0.6767385833021641
Fold 10 r_squared: 0.6901712021309889
Fold 10 Mean of Two Highest Tanimoto Similarities: 0.7664063607869535
Mean Pearson R: 0.41591398065922613
Overall Mean of Two Highest Tanimoto Similarities: 0.772225206112495


In [19]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("cv_results_MLP_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,Cc1ccc(NC(=O)C2CCC2)cc1-c1ccc(C(=O)NCC2CC2)cc1,5.77,5.567621
1,1,Nc1ccccc1Nc1ccc2c(c1)OCc1ccc(OCCN3CCOCC3)cc1C2=O,6.97,7.175056
2,1,O=C(c1c(F)cccc1F)N1CCn2c(cnc2-c2ccc(F)cc2F)C1,6.78,6.876147
3,1,Cc1cccc(-c2nc(-c3ccc(S(C)(=O)=O)cc3)sc2-c2cc[n...,7.47,8.046846
4,1,CC(=O)Nc1ccc(C)c(-c2ccc(C(=O)NCC3CC3)cc2)c1,5.40,5.667792
...,...,...,...,...
13000,10,COC(=O)C(N)CCCCNC(=O)c1ccc2c(c1)C(=O)c1ccc(Nc3...,7.92,7.972514
13001,10,COC(=O)C(N)CNC(=O)c1ccc2c(c1)C(=O)c1ccc(Nc3ccc...,8.27,8.566728
13002,10,CCc1nc(-c2cccc(C)c2)c(-c2cc[nH]c(=NC(=O)Cc3ccc...,8.25,8.385568
13003,10,O=c1ncn2nc(Sc3ccc(F)cc3F)ccc2c1-c1c(Cl)cccc1Cl,7.35,7.229206


In [20]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("cv_results_MLP_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,N#Cc1ccccc1Oc1ccc(C(=O)NCc2ccncn2)cc1,5.55,7.206148
1,1,CC(N=c1nc(N(CC2CCN(S(N)(=O)=O)CC2)C(=O)c2ccc3c...,6.17,6.131078
2,1,Cc1ccc(C(=O)NC2CC2)cc1-c1cc2c[nH]c(=NC3CCOCC3)...,9.05,7.327793
3,1,COc1ccc(N(C(=O)Oc2c(C)cccc2C)c2cc[nH]c(=Nc3ccc...,6.50,5.943081
4,1,Cc1cccc(C)c1OC(=O)N(c1ccc2ccccc2c1)c1cc[nH]c(=...,6.13,6.473256
...,...,...,...,...
1440,10,N=c1c(C(=O)c2ccc(CCCN3CCOCC3)cc2)c[nH]n1-c1ccc...,4.65,5.303539
1441,10,N=c1c(C(=O)c2cccc(-c3cccnc3)c2)c[nH]n1-c1ccc(F...,6.11,6.064291
1442,10,O=C(NC1(c2ccccc2)CC1)c1ccc2c(c1)C(=O)c1ccc(Nc3...,8.60,9.277806
1443,10,CC(C)c1nnc2ccc(-c3c[nH]nc3-c3cc(F)ccc3F)cn12,8.55,7.972504


In [21]:
#RF

In [22]:
import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = RandomForestRegressor(n_estimators=min(25,round(np.sqrt(X_train.shape[0]))), random_state=42, n_jobs=-1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1301, Test size: 144
Fold 1 Accuracy: (0.8972065147929138, 2.931603906725054e-52)
Fold 1 rmse: 0.5746690421934659
Fold 1 r_squared: 0.8008837683616232
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.7783728009562503
Fold 2 - Train size: 1301, Test size: 144
Fold 2 Accuracy: (0.8514557221450321, 1.2272970460394422e-41)
Fold 2 rmse: 0.5859792217590428
Fold 2 r_squared: 0.7133373188107089
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.7706844560884062
Fold 3 - Train size: 1301, Test size: 144
Fold 3 Accuracy: (0.8734790946715043, 3.1220353856031576e-46)
Fold 3 rmse: 0.5777155239848473
Fold 3 r_squared: 0.754800089900393
Fold 3 Mean of Two Highest Tanimoto Similarities: 0.7663866245900152
Fold 4 - Train size: 1301, Test size: 144
Fold 4 Accuracy: (0.8929108024022887, 4.588201911149472e-51)
Fold 4 rmse: 0.526516694628216
Fold 4 r_squared: 0.794919001010512
Fold 4 Mean of Two Highest Tanimoto Similarities: 0.7733275810266682
Fold 5 - Train size: 1301, Test size:

In [23]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("cv_results_RF_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,Cc1ccc(NC(=O)C2CCC2)cc1-c1ccc(C(=O)NCC2CC2)cc1,5.77,5.6488
1,1,Nc1ccccc1Nc1ccc2c(c1)OCc1ccc(OCCN3CCOCC3)cc1C2=O,6.97,7.1168
2,1,O=C(c1c(F)cccc1F)N1CCn2c(cnc2-c2ccc(F)cc2F)C1,6.78,6.9808
3,1,Cc1cccc(-c2nc(-c3ccc(S(C)(=O)=O)cc3)sc2-c2cc[n...,7.47,7.5600
4,1,CC(=O)Nc1ccc(C)c(-c2ccc(C(=O)NCC3CC3)cc2)c1,5.40,5.7792
...,...,...,...,...
13000,10,COC(=O)C(N)CCCCNC(=O)c1ccc2c(c1)C(=O)c1ccc(Nc3...,7.92,7.9012
13001,10,COC(=O)C(N)CNC(=O)c1ccc2c(c1)C(=O)c1ccc(Nc3ccc...,8.27,8.3160
13002,10,CCc1nc(-c2cccc(C)c2)c(-c2cc[nH]c(=NC(=O)Cc3ccc...,8.25,8.1952
13003,10,O=c1ncn2nc(Sc3ccc(F)cc3F)ccc2c1-c1c(Cl)cccc1Cl,7.35,7.1528


In [24]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("cv_results_RF_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,N#Cc1ccccc1Oc1ccc(C(=O)NCc2ccncn2)cc1,5.55,6.4420
1,1,CC(N=c1nc(N(CC2CCN(S(N)(=O)=O)CC2)C(=O)c2ccc3c...,6.17,6.5984
2,1,Cc1ccc(C(=O)NC2CC2)cc1-c1cc2c[nH]c(=NC3CCOCC3)...,9.05,7.6952
3,1,COc1ccc(N(C(=O)Oc2c(C)cccc2C)c2cc[nH]c(=Nc3ccc...,6.50,6.0848
4,1,Cc1cccc(C)c1OC(=O)N(c1ccc2ccccc2c1)c1cc[nH]c(=...,6.13,6.3904
...,...,...,...,...
1440,10,N=c1c(C(=O)c2ccc(CCCN3CCOCC3)cc2)c[nH]n1-c1ccc...,4.65,5.7528
1441,10,N=c1c(C(=O)c2cccc(-c3cccnc3)c2)c[nH]n1-c1ccc(F...,6.11,6.1352
1442,10,O=C(NC1(c2ccccc2)CC1)c1ccc2c(c1)C(=O)c1ccc(Nc3...,8.60,8.1656
1443,10,CC(C)c1nnc2ccc(-c3c[nH]nc3-c3cc(F)ccc3F)cn12,8.55,8.0832


In [25]:
#xgboost

In [26]:
import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = xgb.XGBRegressor(n_estimators=min(25, round(np.sqrt(X_train.shape[0]))), random_state=42, n_jobs=-1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1301, Test size: 144
Fold 1 Accuracy: (0.8919975649992364, 8.110338496482646e-51)
Fold 1 rmse: 0.5931978981672636
Fold 1 r_squared: 0.7878366972140654
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.7783728009562503
Fold 2 - Train size: 1301, Test size: 144
Fold 2 Accuracy: (0.8399935633622191, 1.5675540796749046e-39)
Fold 2 rmse: 0.6003278880615502
Fold 2 r_squared: 0.6991266213331926
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.7706844560884062
Fold 3 - Train size: 1301, Test size: 144
Fold 3 Accuracy: (0.856803033572274, 1.108511267586913e-42)
Fold 3 rmse: 0.6056379257217677
Fold 3 r_squared: 0.7305250762096489
Fold 3 Mean of Two Highest Tanimoto Similarities: 0.7663866245900152
Fold 4 - Train size: 1301, Test size: 144
Fold 4 Accuracy: (0.9098208528160788, 4.250712418775026e-56)
Fold 4 rmse: 0.4993578308911225
Fold 4 r_squared: 0.8155303764244213
Fold 4 Mean of Two Highest Tanimoto Similarities: 0.7733275810266682
Fold 5 - Train size: 1301, Test size

In [27]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("cv_results_xgboost_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,Cc1ccc(NC(=O)C2CCC2)cc1-c1ccc(C(=O)NCC2CC2)cc1,5.77,5.837232
1,1,Nc1ccccc1Nc1ccc2c(c1)OCc1ccc(OCCN3CCOCC3)cc1C2=O,6.97,7.284721
2,1,O=C(c1c(F)cccc1F)N1CCn2c(cnc2-c2ccc(F)cc2F)C1,6.78,7.480220
3,1,Cc1cccc(-c2nc(-c3ccc(S(C)(=O)=O)cc3)sc2-c2cc[n...,7.47,7.661768
4,1,CC(=O)Nc1ccc(C)c(-c2ccc(C(=O)NCC3CC3)cc2)c1,5.40,6.053919
...,...,...,...,...
13000,10,COC(=O)C(N)CCCCNC(=O)c1ccc2c(c1)C(=O)c1ccc(Nc3...,7.92,8.103864
13001,10,COC(=O)C(N)CNC(=O)c1ccc2c(c1)C(=O)c1ccc(Nc3ccc...,8.27,8.131000
13002,10,CCc1nc(-c2cccc(C)c2)c(-c2cc[nH]c(=NC(=O)Cc3ccc...,8.25,7.892226
13003,10,O=c1ncn2nc(Sc3ccc(F)cc3F)ccc2c1-c1c(Cl)cccc1Cl,7.35,6.966002


In [28]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("cv_results_xgboost_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,N#Cc1ccccc1Oc1ccc(C(=O)NCc2ccncn2)cc1,5.55,6.439711
1,1,CC(N=c1nc(N(CC2CCN(S(N)(=O)=O)CC2)C(=O)c2ccc3c...,6.17,6.811404
2,1,Cc1ccc(C(=O)NC2CC2)cc1-c1cc2c[nH]c(=NC3CCOCC3)...,9.05,7.750926
3,1,COc1ccc(N(C(=O)Oc2c(C)cccc2C)c2cc[nH]c(=Nc3ccc...,6.50,6.547199
4,1,Cc1cccc(C)c1OC(=O)N(c1ccc2ccccc2c1)c1cc[nH]c(=...,6.13,6.547199
...,...,...,...,...
1440,10,N=c1c(C(=O)c2ccc(CCCN3CCOCC3)cc2)c[nH]n1-c1ccc...,4.65,5.796494
1441,10,N=c1c(C(=O)c2cccc(-c3cccnc3)c2)c[nH]n1-c1ccc(F...,6.11,5.784060
1442,10,O=C(NC1(c2ccccc2)CC1)c1ccc2c(c1)C(=O)c1ccc(Nc3...,8.60,8.095254
1443,10,CC(C)c1nnc2ccc(-c3c[nH]nc3-c3cc(F)ccc3F)cn12,8.55,8.062097
